In [ ]:
# RICORDATI DI CONTROLLARE IL PROBLEMA DEL DISALLINEAMENTO TRA G_pruned_nodes_final.csv E GLI screen_name EFFETTIVAMENTE CARICATI IN QUESTO NOTEBOOK!!!

In [72]:
# !pip install treetaggerwrapper

In [111]:
import json, time
import pandas as pd
import treetaggerwrapper
import string
from datetime import datetime
import pyLDAvis.sklearn

import warnings
warnings.filterwarnings("ignore")

In [109]:
pyLDAvis.enable_notebook()

### Carichiamo la partizione

In [3]:
with open('Partition_pruned_EN.json') as json_file:
    partition = json.load(json_file)

In [4]:
partition_df = pd.DataFrame.from_dict(partition, orient='index')
partition_df = partition_df.reset_index()
partition_df.columns = ['screen_name','community_id']
partition_df

,screen_name,community_id
0,100REScom,6
1,100erneuerbar,6
2,100givers,2
3,2019Biomass,2
4,2020_baseload,1
...,...,...
3137,younousomarjee,4
3138,zabala_eu,2
3139,zeekayinfo3,3
3140,zlegrand,2


### Carichiamo le descrizioni degli utenti

In [5]:
seed = pd.read_csv('seed_incompleto.csv') #Mancano innovatec_spa e italgas
seed = seed.drop(['Unnamed: 0'],axis=1)
seed = seed.rename(columns={"index": "screen_name"})
seed = seed.drop(['followers_count_high_values', 'followers_count_medium_high_values', 'followers_count_medium_low_values', 'followers_count_low_values'],axis=1)
innovatec_spa = {
    'screen_name': 'innovatec_spa',
    'location': 'Milano',
    'friends_count': 282,
    'listed_count': None,
    'followers_count': 2052,
    'created_at': 'Thu May 01 00:00:00 +0000 2014',
    'lang': None,
    'statuses_count': None,
    'description': 'Operiamo nel settore delle energie rinnovabili offrendo soluzioni tecnologicamente all’avanguardia per la produzione, la gestione e la distribuzione di energia.'
}
italgas = {
    'screen_name': 'italgas',
    'location': 'Milano, Lombardia',
    'friends_count': 50,
    'listed_count': 8,
    'followers_count': 2543,
    'created_at': 'Sun Apr 18 13:42:27 +0000 2016',
    'lang': None,
    'statuses_count': None,
    'description': 'Da oltre 180 anni, siamo leader in Italia nella distribuzione del gas naturale. Since 1837, Italian leader in the distribution of natural gas.'
}
seed = seed.append([innovatec_spa,italgas], ignore_index=True)


friends_1st_level = pd.read_csv('data/friends_1st_level_filtered.csv')
friends_1st_level = friends_1st_level.drop(['Unnamed: 0','energy', 'label'],axis=1)

friends_2nd_level = pd.read_csv('friends_2nd_level_EN.csv')
friends_2nd_level = friends_2nd_level.rename(columns={"Unnamed: 0": "screen_name"})
# friends_2nd_level = friends_2nd_level.drop(['followers_count_high_values', 'followers_count_medium_high_values', 'followers_count_medium_low_values', 'followers_count_low_values', 'pred'],axis=1)

In [6]:
seed.shape

(14, 9)

In [7]:
friends_1st_level.shape

(740, 9)

In [8]:
friends_2nd_level.shape

(4620, 11)

In [9]:
all_users = pd.concat([seed, friends_1st_level, friends_2nd_level])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
all_users.shape

(5374, 11)

In [11]:
all_users = all_users.drop_duplicates(subset='screen_name', keep="first")

### Facciamo la join per associare la partizione al dataframe di tutti gli utenti

In [12]:
all_users_part = all_users.merge(partition_df, left_on='screen_name', right_on='screen_name')

In [13]:
all_users_part

,created_at,description,detected_lang,followers_count,friends_count,lang,listed_count,location,pred,screen_name,statuses_count,community_id
0,Tue Mar 13 08:49:40 +0000 2012,"We produce wind and solar energy, biomass, and...",NaN,1929.0,312.0,NaN,16.0,NaN,NaN,falckrenewables,695,0
1,Tue Mar 23 16:14:19 +0000 2010,Diventiamo l’#energiachecambiatutto. Anche con...,NaN,10235.0,1601.0,NaN,295.0,"Foro Buonaparte 31, Milano",NaN,EdisonNews,12836,0
2,Fri Mar 06 20:20:52 +0000 2009,Profilo ufficiale in italiano di #Enel. Seguic...,NaN,26703.0,5893.0,NaN,293.0,NaN,NaN,EnelGroupIT,19668,0
3,Thu Jun 30 10:52:29 +0000 2011,La responsabilità dell'energia. L'energia dell...,NaN,5238.0,686.0,NaN,94.0,Italia,NaN,TernaSpA,2093,0
4,Thu Oct 25 13:40:33 +0000 2012,Landirenzo: leader nella produzione di impiant...,NaN,621.0,149.0,NaN,23.0,Reggio Emilia,NaN,LandirenzoSpA,1310,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Thu Apr 25 11:44:24 +0000 2013,"Working at the European Commission, DG Environ...",en,21.0,19.0,NaN,2.0,Brussels,1.0,c_fenech,6,4
3138,Thu Mar 19 16:02:36 +0000 2009,"Rapporteur UN CEPA, Coord.Environm. Impl. Revi...",en,466.0,275.0,NaN,21.0,Brussel,1.0,LouisMeuleman,452,4
3139,Sat Jul 28 11:21:02 +0000 2012,WasteReuse European-funded project aims to the...,en,187.0,222.0,NaN,14.0,Brussels,1.0,WasteReuse,543,4
3140,Tue Nov 27 15:32:21 +0000 2012,Making the food chain more efficient to produc...,en,447.0,795.0,NaN,21.0,NaN,1.0,RESFOODProject,812,4


## Carichiamo i testi dei tweet degli utenti

In [14]:
from nltk.corpus import stopwords
import re

stop_words = stopwords.words('english')

def processTweet(tweet):
    """
    Takes in a string of text, then performs the following:
    1. Removes links, special characters and other bulk cleaning
    2. Returns the tweet text cleaned
    """
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    #tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    #tweet = re.sub(r'#\w*', '', tweet)
    # Remove words with 2 or fewer letters
    tweet = re.sub(r'\b\w\b', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove new line character
    tweet = tweet.replace('\n', ' ')
    # Remove …
    tweet = tweet.replace('…', ' ')
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ')
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF')
    return tweet


In [45]:
import json

start_date = "2020-01-06"
end_date = "2020-01-13"

timelines_dir = "output4_G_pruned/"

tweets = {}
for us in all_users['screen_name']:
    filename = timelines_dir+"timeline_"+us+".json"
    print(filename, end=" ")
    data = []
    try:
        with open(filename) as json_file:
            for js in json_file:
                tw_time = json.loads(js)['tweet']['created_at']
                tw_datetime = datetime.strftime(datetime.strptime(tw_time,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
                if tw_datetime>=start_date and tw_datetime<end_date:
                    data.append(processTweet(json.loads(js)['tweet']['text']))
                #break
        print(len(data))
        tweets[us] = data
        #break
    except:
        print("NO DATA FOR",us)

output4_G_pruned/timeline_falckrenewables.json 1
output4_G_pruned/timeline_EdisonNews.json 0
output4_G_pruned/timeline_EnelGroupIT.json 9
output4_G_pruned/timeline_TernaSpA.json 4
output4_G_pruned/timeline_LandirenzoSpA.json 2
output4_G_pruned/timeline_eni.json 19
output4_G_pruned/timeline_snam.json 6
output4_G_pruned/timeline_ERGnow.json 0
output4_G_pruned/timeline_GruppoHera.json 31
output4_G_pruned/timeline_gruppoiren.json 6
output4_G_pruned/timeline_gruppo_a2a.json 8
output4_G_pruned/timeline_cirgroup.json NO DATA FOR cirgroup
output4_G_pruned/timeline_innovatec_spa.json 0
output4_G_pruned/timeline_italgas.json 5
output4_G_pruned/timeline_intel.json NO DATA FOR intel
output4_G_pruned/timeline_GretaThunberg.json 59
output4_G_pruned/timeline_MichaelPhelps.json NO DATA FOR MichaelPhelps
output4_G_pruned/timeline_BBCScienceNews.json 30
output4_G_pruned/timeline_NYTScience.json 0
output4_G_pruned/timeline_ftchina.json NO DATA FOR ftchina
output4_G_pruned/timeline_UNEP.json 36
output4_G_

output4_G_pruned/timeline_Solar_Sister.json 7
output4_G_pruned/timeline_oppgreen.json 0
output4_G_pruned/timeline_WMBtweets.json 55
output4_G_pruned/timeline_tuttogreen.json 16
output4_G_pruned/timeline_FerdiGiugliano.json 4
output4_G_pruned/timeline_AEEnet.json 24
output4_G_pruned/timeline_jeffmcstras.json NO DATA FOR jeffmcstras
output4_G_pruned/timeline_altromercato.json NO DATA FOR altromercato
output4_G_pruned/timeline_WWFScotland.json 19
output4_G_pruned/timeline_ssilvia13.json NO DATA FOR ssilvia13
output4_G_pruned/timeline_mitenergy.json 10
output4_G_pruned/timeline_WindEnergyPower.json NO DATA FOR WindEnergyPower
output4_G_pruned/timeline_GreenMaven.json 0
output4_G_pruned/timeline_BloombergNRG.json 167
output4_G_pruned/timeline_CabotOG.json NO DATA FOR CabotOG
output4_G_pruned/timeline_EP_Environment.json 1
output4_G_pruned/timeline_Worldwatch.json 2
output4_G_pruned/timeline_timothybhurst.json 5
output4_G_pruned/timeline_rep_ambiente.json NO DATA FOR rep_ambiente
output4_G_p

output4_G_pruned/timeline_GreenUnivers.json 80
output4_G_pruned/timeline_antosavarese.json 0
output4_G_pruned/timeline_RevEnergia.json 8
output4_G_pruned/timeline_MyGreenSide.json NO DATA FOR MyGreenSide
output4_G_pruned/timeline_NUTROBrand.json NO DATA FOR NUTROBrand
output4_G_pruned/timeline_EcoEcologia.json NO DATA FOR EcoEcologia
output4_G_pruned/timeline_sciencetargets.json 26
output4_G_pruned/timeline_avoicomunicare.json NO DATA FOR avoicomunicare
output4_G_pruned/timeline_EuropeanGasHub.json 6
output4_G_pruned/timeline_EcoSpiragli.json NO DATA FOR EcoSpiragli
output4_G_pruned/timeline_premiomarzotto.json NO DATA FOR premiomarzotto
output4_G_pruned/timeline_bluerating_com.json NO DATA FOR bluerating_com
output4_G_pruned/timeline_euwidenergie.json 16
output4_G_pruned/timeline_GiaSilvestrini.json 100
output4_G_pruned/timeline_IWEA.json 26
output4_G_pruned/timeline_andrew_ward1.json NO DATA FOR andrew_ward1
output4_G_pruned/timeline_CE_NatGas.json 12
output4_G_pruned/timeline_brywor

output4_G_pruned/timeline_canaleenergia.json 30
output4_G_pruned/timeline_SmartEnergyExpo.json 0
output4_G_pruned/timeline_fotovoltaicinfo.json NO DATA FOR fotovoltaicinfo
output4_G_pruned/timeline_ValoreSociale.json NO DATA FOR ValoreSociale
output4_G_pruned/timeline_Earth_NewsRT.json NO DATA FOR Earth_NewsRT
output4_G_pruned/timeline_Envirocruit.json NO DATA FOR Envirocruit
output4_G_pruned/timeline_jyn8101.json NO DATA FOR jyn8101
output4_G_pruned/timeline_stefanobesana.json NO DATA FOR stefanobesana
output4_G_pruned/timeline_markettara.json NO DATA FOR markettara
output4_G_pruned/timeline_G20Climate.json 1
output4_G_pruned/timeline_AfricaEUEnergy.json 1
output4_G_pruned/timeline_ap_legambiente.json 70
output4_G_pruned/timeline_dariussnieckus.json 43
output4_G_pruned/timeline_IPIECA.json 0
output4_G_pruned/timeline_windbizfactor.json NO DATA FOR windbizfactor
output4_G_pruned/timeline_Urbact_IT.json NO DATA FOR Urbact_IT
output4_G_pruned/timeline_EarthDayItalia.json NO DATA FOR Eart

output4_G_pruned/timeline_wefenergy.json 8
output4_G_pruned/timeline_amicidellaterra.json NO DATA FOR amicidellaterra
output4_G_pruned/timeline_LegambienteUmbr.json NO DATA FOR LegambienteUmbr
output4_G_pruned/timeline_investtrentino.json NO DATA FOR investtrentino
output4_G_pruned/timeline_greengraffiti.json NO DATA FOR greengraffiti
output4_G_pruned/timeline_Comonext_.json NO DATA FOR Comonext_
output4_G_pruned/timeline__amblav.json NO DATA FOR _amblav
output4_G_pruned/timeline_EnergyViewpoint.json 0
output4_G_pruned/timeline_RANRoberto.json NO DATA FOR RANRoberto
output4_G_pruned/timeline_rvlandberg.json 25
output4_G_pruned/timeline_AigetEnergia.json NO DATA FOR AigetEnergia
output4_G_pruned/timeline_FORESIGHTdk.json 6
output4_G_pruned/timeline_kaitaniemi.json NO DATA FOR kaitaniemi
output4_G_pruned/timeline_aristotele30.json NO DATA FOR aristotele30
output4_G_pruned/timeline_PicloEnergy.json 0
output4_G_pruned/timeline_EnvComm_ACT.json NO DATA FOR EnvComm_ACT
output4_G_pruned/timel

...

output4_G_pruned/timeline_GreaterNicosia.json 0
output4_G_pruned/timeline_KatriinaPajari.json 11
output4_G_pruned/timeline_CEEREAL_EU.json 2
output4_G_pruned/timeline_Europatat.json 0
output4_G_pruned/timeline_CUEBusiness.json 6
output4_G_pruned/timeline_vozemberg.json 18
output4_G_pruned/timeline_wcarrara.json 7
output4_G_pruned/timeline_ACTRISRI.json 4
output4_G_pruned/timeline_D_Scallan.json 0
output4_G_pruned/timeline_boddegal.json 0
output4_G_pruned/timeline_GiacomoFassina.json 6
output4_G_pruned/timeline_HistoriaEuropa.json 3
output4_G_pruned/timeline_AlexNicandrou.json 0
output4_G_pruned/timeline_EP_Romania.json 5
output4_G_pruned/timeline_MihCalin.json 1
output4_G_pruned/timeline_GKremlis.json 3
output4_G_pruned/timeline_MattiaSecchi91.json 4
output4_G_pruned/timeline_Dieterstaat.json 1
output4_G_pruned/timeline_AWintergroup.json 5
output4_G_pruned/timeline_liferefibre.json 0
output4_G_pruned/timeline_LIFEALGAECAN.json 0
output4_G_pruned/timeline_ChIMERA_med.json 0
output4_G_pr

output4_G_pruned/timeline_Frontex.json 10
output4_G_pruned/timeline_JudithPoser.json NO DATA FOR JudithPoser
output4_G_pruned/timeline_endless_europe.json NO DATA FOR endless_europe
output4_G_pruned/timeline_SamuelAgini.json NO DATA FOR SamuelAgini
output4_G_pruned/timeline_ThoenissenC.json NO DATA FOR ThoenissenC
output4_G_pruned/timeline_BenDummett.json 15
output4_G_pruned/timeline_josieleblond.json NO DATA FOR josieleblond
output4_G_pruned/timeline_sivieroand.json NO DATA FOR sivieroand
output4_G_pruned/timeline_cbarnard_IDC.json NO DATA FOR cbarnard_IDC
output4_G_pruned/timeline_BoLykkegaard.json NO DATA FOR BoLykkegaard
output4_G_pruned/timeline_SmartGridPro.json NO DATA FOR SmartGridPro
output4_G_pruned/timeline_deantchuang.json 0
output4_G_pruned/timeline_NUCLEARPOWEREUR.json NO DATA FOR NUCLEARPOWEREUR
output4_G_pruned/timeline_milankalal.json NO DATA FOR milankalal
output4_G_pruned/timeline_SolarECycles.json NO DATA FOR SolarECycles
output4_G_pruned/timeline_geoffrey_manley.js

output4_G_pruned/timeline_EURightsAgency.json 7
output4_G_pruned/timeline_LIFE_CleanUp.json 0
output4_G_pruned/timeline_GREENOMED_EU.json 2
output4_G_pruned/timeline_MedfestProject.json 0
output4_G_pruned/timeline_EurOBIS_VLIZ.json 1
output4_G_pruned/timeline_EULawDataPubs.json 27
output4_G_pruned/timeline_eurocrowd.json 0
output4_G_pruned/timeline_EESC_INT.json 3
output4_G_pruned/timeline_EU_procure.json 0
output4_G_pruned/timeline_PERMIDES_EU.json 0
output4_G_pruned/timeline_AurelieGom.json 0
output4_G_pruned/timeline_EVBN_Project.json 0
output4_G_pruned/timeline_Clusters_EU.json 14
output4_G_pruned/timeline_EUdigitalsme.json 7
output4_G_pruned/timeline_GearingUp_.json 1
output4_G_pruned/timeline_EDC_UCY.json 0
output4_G_pruned/timeline_EDC_UNic.json 0
output4_G_pruned/timeline_NicolasBrookes.json 6
output4_G_pruned/timeline_EA_CoR.json 5
output4_G_pruned/timeline_Claude_Moraes.json 2
output4_G_pruned/timeline_EP_Justice.json 19
output4_G_pruned/timeline_EU_Justice.json 2
output4_G_p

output4_G_pruned/timeline_alberto_cottica.json 33
output4_G_pruned/timeline_albertomz.json NO DATA FOR albertomz
output4_G_pruned/timeline_hpkeymark.json NO DATA FOR hpkeymark
output4_G_pruned/timeline_Herve_Amosse.json NO DATA FOR Herve_Amosse
output4_G_pruned/timeline_AlliJCohen.json NO DATA FOR AlliJCohen
output4_G_pruned/timeline_SFranconville.json NO DATA FOR SFranconville
output4_G_pruned/timeline_jan_masschelin.json NO DATA FOR jan_masschelin
output4_G_pruned/timeline_soniadupont7.json NO DATA FOR soniadupont7
output4_G_pruned/timeline_barbaravermeire.json NO DATA FOR barbaravermeire
output4_G_pruned/timeline_flaviofacioni.json NO DATA FOR flaviofacioni
output4_G_pruned/timeline_EPSAndy.json NO DATA FOR EPSAndy
output4_G_pruned/timeline_Energy_Yield.json NO DATA FOR Energy_Yield
output4_G_pruned/timeline_Inaelgroup.json NO DATA FOR Inaelgroup
output4_G_pruned/timeline_SGJEvers.json NO DATA FOR SGJEvers
output4_G_pruned/timeline_EnergyGuyPhilly.json NO DATA FOR EnergyGuyPhilly
ou

output4_G_pruned/timeline_p1020.json 1
output4_G_pruned/timeline_MikeDPeacock.json NO DATA FOR MikeDPeacock
output4_G_pruned/timeline_londonjourno.json NO DATA FOR londonjourno
output4_G_pruned/timeline_reutersHuwJ.json 4
output4_G_pruned/timeline_ReutersAdrianC.json NO DATA FOR ReutersAdrianC
output4_G_pruned/timeline_KutoaLtd.json NO DATA FOR KutoaLtd
output4_G_pruned/timeline_CanadaEUTrade.json NO DATA FOR CanadaEUTrade
output4_G_pruned/timeline_baronsauvage.json NO DATA FOR baronsauvage
output4_G_pruned/timeline_PvanL23.json NO DATA FOR PvanL23
output4_G_pruned/timeline_lucianSF.json NO DATA FOR lucianSF
output4_G_pruned/timeline_Siby1966.json NO DATA FOR Siby1966
output4_G_pruned/timeline_LibDemMEPs.json NO DATA FOR LibDemMEPs
output4_G_pruned/timeline_miningimpact.json NO DATA FOR miningimpact
output4_G_pruned/timeline_danobrien20.json NO DATA FOR danobrien20
output4_G_pruned/timeline_gridinterop.json NO DATA FOR gridinterop
output4_G_pruned/timeline_bromotion.json NO DATA FOR br

In [46]:
len(tweets.keys())

3112

In [47]:
cnt = 0
for k,v in tweets.items():
    cnt+=len(v)
print(cnt)

28588


In [48]:
users_comm = {}
for index,row in all_users_part[['screen_name','community_id']].iterrows():
    users_comm[row['screen_name']] = row['community_id']
print(users_comm)

{'falckrenewables': 0, 'EdisonNews': 0, 'EnelGroupIT': 0, 'TernaSpA': 0, 'LandirenzoSpA': 0, 'eni': 0, 'snam': 5, 'ERGnow': 0, 'GruppoHera': 0, 'gruppoiren': 0, 'gruppo_a2a': 3, 'innovatec_spa': 0, 'italgas': 0, 'GretaThunberg': 0, 'BBCScienceNews': 0, 'NYTScience': 0, 'UNEP': 0, 'Total': 5, 'ENERGY': 0, 'UNFCCC': 1, 'PaoloGentiloni': 5, 'EnvAgency': 0, 'ClimateReality': 2, 'guardianeco': 3, 'HuffPostGreen': 0, 'USATODAYmoney': 0, 'Chevron': 3, 'NBCNewsBusiness': 0, 'PlanetGreen': 3, 'climateprogress': 3, 'OGJOnline': 5, 'IPCC_CH': 5, 'EnvDefenseFund': 3, 'IEA': 0, 'ClimateDesk': 3, 'Connect4Climate': 2, 'ClimateGroup': 5, 'EIAgov': 0, 'REWorld': 6, 'WorldOil': 3, 'insideclimate': 3, 'TheGlobalGoals': 0, 'GreenEnergyNews': 3, 'GreenBiz': 3, 'DeSmogBlog': 3, 'SolarEnergyNews': 0, 'ftenergy': 0, 'SEIA': 3, 'EU_ENV': 4, 'arikring': 2, 'GreenNewsDaily': 3, 'nytclimate': 0, 'IRENA': 0, 'EUEnvironment': 1, 'AWEA': 3, 'PPFEnergia': 3, 'APIenergy': 0, 'resilientplanb': 1, 'EnergyCollectiv': 3,

In [49]:
tweets_partitioned = {}

for k,v in tweets.items():
    if users_comm[k] in tweets_partitioned.keys():
        tmp = tweets_partitioned[users_comm[k]]
        tmp.append(v)
        tweets_partitioned[users_comm[k]] = tmp
    else:
        tweets_partitioned[users_comm[k]] = [v]

In [50]:
tweets_partitioned.keys()

dict_keys([0, 5, 3, 1, 2, 6, 4, 7])

In [51]:
for k,v in tweets_partitioned.items():
    print(k,len(v))

0 98
5 358
3 471
1 173
2 496
6 360
4 1066
7 90


In [52]:
treetaggerwrapper.enable_debugging_log()

In [53]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="en") #, TAGOPT="--debug")

T140126001977152 2020-06-24 10:04:20,925 DEBUG Using treetaggerwrapper.py from /opt/conda/lib/python3.7/site-packages/treetaggerwrapper.py
T140126001977152 2020-06-24 10:04:20,926 DEBUG Found param TAGLANG in paramsdict.
T140126001977152 2020-06-24 10:04:20,926 INFO lang=en
T140126001977152 2020-06-24 10:04:20,927 DEBUG Use default value for param TAGDIR.
T140126001977152 2020-06-24 10:04:20,927 INFO Use previously found TreeTagger directory: /home/vincenzo/tree-tagger-linux
T140126001977152 2020-06-24 10:04:20,928 INFO tagdir=/home/vincenzo/tree-tagger-linux
T140126001977152 2020-06-24 10:04:20,928 INFO tagbin=/home/vincenzo/tree-tagger-linux/bin/tree-tagger
T140126001977152 2020-06-24 10:04:20,929 DEBUG Use default value for param TAGPARFILE.
T140126001977152 2020-06-24 10:04:20,930 INFO tagparfile=/home/vincenzo/tree-tagger-linux/lib/english.par
T140126001977152 2020-06-24 10:04:20,930 DEBUG Use default value for param TAGINENC.
T140126001977152 2020-06-24 10:04:20,931 DEBUG Use def

In [54]:
tweets_partitioned_as_strings = {}

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

# for k,v in test_dict.items():
for k,v in tweets_partitioned.items():
    s = ""
    for i in v: # v e' la lista di stringhe di un certo utente
        s += ' '.join(i)
        s += ' '
    s = s.lower()
    s = s.translate(translator)
    time.sleep(2)
    tags = tagger.tag_text(s)
    tagged_words = treetaggerwrapper.make_tags(tags)
    #print(tagged_words[:1000])
    s_tmp = ""
    for tw in tagged_words:
        if type(tw) != treetaggerwrapper.NotTag and len(tw[2]) > 3:
            #print(tw[2])
            #break
            s_tmp += tw[2] + " "
#     print(s_tmp[:1000])
#     break
    tweets_partitioned_as_strings[k] = s_tmp
    
# tweets_partitioned_as_strings

T140126001977152 2020-06-24 10:04:22,965 DEBUG tag_text with option: numlines=0, tagonly=0, prepronly=0, notagurl=0, tagblanks=0, notagemail=0, notagip=0, notagdns=0, nosgmlsplit=0).
T140126001977152 2020-06-24 10:04:22,966 DEBUG Pre-processing text with internal chunker.
T140126001977152 2020-06-24 10:04:22,967 DEBUG Preparing text for tagger with options tagblanks=0, numlines=0, notagurl=0, notagemail=0, notagip=0, notagdns=0, nosgmlsplit=0).
T140126001977152 2020-06-24 10:04:22,968 DEBUG Identifying SGML tags from within text.
T140126001977152 2020-06-24 10:04:22,968 DEBUG Splitted between SGML tags and others %r.
T140126001977152 2020-06-24 10:04:22,969 DEBUG Replacing blanks by spaces.
T140126001977152 2020-06-24 10:04:22,989 DEBUG Blanks replacement done.
T140126001977152 2020-06-24 10:04:22,990 DEBUG Replacing URLs.
T140126001977152 2020-06-24 10:04:23,016 DEBUG URLs replacement done.
T140126001977152 2020-06-24 10:04:23,016 DEBUG Replacing Emails.
T140126001977152 2020-06-24 10

T140126001977152 2020-06-24 10:04:35,515 DEBUG Identifying SGML tags from within text.
T140126001977152 2020-06-24 10:04:35,516 DEBUG Splitted between SGML tags and others %r.
T140126001977152 2020-06-24 10:04:35,516 DEBUG Replacing blanks by spaces.
T140126001977152 2020-06-24 10:04:35,560 DEBUG Blanks replacement done.
T140126001977152 2020-06-24 10:04:35,560 DEBUG Replacing URLs.
T140126001977152 2020-06-24 10:04:35,622 DEBUG URLs replacement done.
T140126001977152 2020-06-24 10:04:35,623 DEBUG Replacing Emails.
T140126001977152 2020-06-24 10:04:35,667 DEBUG Emails replacement done.
T140126001977152 2020-06-24 10:04:35,668 DEBUG Replacing IP addresses.
T140126001977152 2020-06-24 10:04:35,694 DEBUG IP adresses replacement done.
T140126001977152 2020-06-24 10:04:35,694 DEBUG Replacing DNS names.
T140126001977152 2020-06-24 10:04:35,792 DEBUG DNS names replacement done.
T140126001977152 2020-06-24 10:04:35,793 DEBUG Splittint parts of text.
T140126001977152 2020-06-24 10:04:36,155 DEB

In [55]:
# src: https://github.com/stopwords-iso/stopwords-it/blob/master/stopwords-it.json
stop_words_it = []

lettere_accentate = {
    'à': 'a',
    'è': 'e',
    'ì': 'i',
    'ò': 'o',
    'ù': 'u',
    'é': 'e',
    'ó': 'o',
    'í': 'i'
}

with open("stopwords-it.json") as json_file:  
    stop_words_it = json.load(json_file)

for i in stop_words_it:
    for acc in lettere_accentate:
        if acc in i:
            # aggiunge alle stop word con l'accento le parole equivalenti senza l'accento
            stop_words_it.append(i.replace(acc,lettere_accentate[acc]))
    
stop_words_it = stop_words_it+ ['goo', 'http', 'https', 'com', 'www', 'htm','html', 'php']

In [56]:
# src: https://gist.github.com/sebleier/554280
stop_words_en = []
file = open("stopwords-en.txt")
while 1:
    line = file.readline()
    if not line: break
    stop_words_en.append(line.split()[0])

In [57]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words=stop_words_it+stop_words_en, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<ipython-input-57-cde1dcb87f26>:6: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [58]:
# TOPICS OF ALL COMMUNITIES

panel_to_save_coll = {}

for k in tweets_partitioned.keys():
    
    data_vectorized = vectorizer.fit_transform(tweets_partitioned_as_strings[k].split())

    NUM_TOPICS = 3

    # Build a Latent Dirichlet Allocation Model
    lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=50, learning_method='online')
    lda_Z = lda_model.fit_transform(data_vectorized)

    # IL NUMERO DI DOCUMENTI QUI INTENDE IL NUMERO DI PAROLE
    print("NumDocuments,NumTopics:",lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS) 

    topics = {}
    for idx, topic in enumerate(lda_model.components_):
        topics[idx] = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-NUM_TOPICS - 1:-1]]

    panel_to_save = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

    panel_to_save_coll[k] = panel_to_save

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ahim', 'avr', 'citt', 'dovr', 'far', 'perch', 'perci', 'sar', 'star'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


NumDocuments,NumTopics: (25298, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


NumDocuments,NumTopics: (52466, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


...

/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


NumDocuments,NumTopics: (71266, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


NumDocuments,NumTopics: (10332, 3)


/home/vincenzo/.local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [63]:
panel_to_save_coll[0]

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  33.424594
1      -387.996002  -373.316162       2        1  33.322525
0     -2184.745605   -60.565800       3        1  33.252881, topic_info=    Category        Freq             Term       Total  loglift  logprob
825  Default  438.000000            solar  438.000000  30.0000  30.0000
726  Default  379.000000             read  379.000000  29.0000  29.0000
277  Default  256.000000           energy  256.000000  28.0000  28.0000
743  Default  138.000000  renewableenergy  138.000000  27.0000  27.0000
473  Default  122.000000            irena  122.000000  26.0000  26.0000
..       ...         ...              ...         ...      ...      ...
271   Topic3   23.513662         electric   24.143173   1.0746  -5.2908
480   Topic3   23.508190          january   24.138241   1.0746  -5.2911
446   Topic3   23.349427            india   23.979013   1.0744  -5.2979
520   Topic3   22.715893             like   23.345606   1.0737  -5.3254
388   Topic3   22.676477       government   23.306360   1.0736  -5.3271

[120 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
13        2  0.975761     action
35        1  0.989581   american
39        1  0.989224   announce
62        1  0.977396  australia
72        3  0.981650       back
...     ...       ...        ...
984       3  0.987982       week
998       1  0.987609       work
1002      3  0.997286      world
1004      3  0.978166      would
1007      2  0.991290       year

[90 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

## Analisi dei soli hashtag

In [ ]:
import json

start_date = "2020-01-06"
end_date = "2020-01-13"

timelines_dir = "output4_G_pruned/"

tweets_ht = {}
for us in all_users['screen_name']:
    filename = timelines_dir+"timeline_"+us+".json"
    print(filename, end=" ")
    hashtags = []
    try:
        with open(filename) as json_file:
            for js in json_file:
                tw_time = json.loads(js)['tweet']['created_at']
                tw_datetime = datetime.strftime(datetime.strptime(tw_time,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
                if tw_datetime>=start_date and tw_datetime<end_date:
                    for e in json.loads(js)['tweet']['entities']['hashtags']:
                        hashtags.append('#'+e['text'].lower())
                #break
        print(len(hashtags))
        tweets_ht[us] = hashtags
        #break
    except:
        print("NO DATA FOR",us)

output4_G_pruned/timeline_falckrenewables.json 1
output4_G_pruned/timeline_EdisonNews.json 0
output4_G_pruned/timeline_EnelGroupIT.json 17
output4_G_pruned/timeline_TernaSpA.json 4
output4_G_pruned/timeline_LandirenzoSpA.json 2
output4_G_pruned/timeline_eni.json 14
output4_G_pruned/timeline_snam.json 3
output4_G_pruned/timeline_ERGnow.json 0
output4_G_pruned/timeline_GruppoHera.json 37
output4_G_pruned/timeline_gruppoiren.json 16
output4_G_pruned/timeline_gruppo_a2a.json 1
output4_G_pruned/timeline_cirgroup.json NO DATA FOR cirgroup
output4_G_pruned/timeline_innovatec_spa.json 0
output4_G_pruned/timeline_italgas.json 6
output4_G_pruned/timeline_intel.json NO DATA FOR intel
output4_G_pruned/timeline_GretaThunberg.json 46
output4_G_pruned/timeline_MichaelPhelps.json NO DATA FOR MichaelPhelps
output4_G_pruned/timeline_BBCScienceNews.json 0
output4_G_pruned/timeline_NYTScience.json 0
output4_G_pruned/timeline_ftchina.json NO DATA FOR ftchina
output4_G_pruned/timeline_UNEP.json 18
output4_G

output4_G_pruned/timeline_Solar_Sister.json 9
output4_G_pruned/timeline_oppgreen.json 0
output4_G_pruned/timeline_WMBtweets.json 31
output4_G_pruned/timeline_tuttogreen.json 0
output4_G_pruned/timeline_FerdiGiugliano.json 0
output4_G_pruned/timeline_AEEnet.json 16
output4_G_pruned/timeline_jeffmcstras.json NO DATA FOR jeffmcstras
output4_G_pruned/timeline_altromercato.json NO DATA FOR altromercato
output4_G_pruned/timeline_WWFScotland.json 2
output4_G_pruned/timeline_ssilvia13.json NO DATA FOR ssilvia13
output4_G_pruned/timeline_mitenergy.json 1
output4_G_pruned/timeline_WindEnergyPower.json NO DATA FOR WindEnergyPower
output4_G_pruned/timeline_GreenMaven.json 0
output4_G_pruned/timeline_BloombergNRG.json 20
output4_G_pruned/timeline_CabotOG.json NO DATA FOR CabotOG
output4_G_pruned/timeline_EP_Environment.json 0
output4_G_pruned/timeline_Worldwatch.json 0
output4_G_pruned/timeline_timothybhurst.json 0
output4_G_pruned/timeline_rep_ambiente.json NO DATA FOR rep_ambiente
output4_G_prune

output4_G_pruned/timeline_GreenUnivers.json 55
output4_G_pruned/timeline_antosavarese.json 0
output4_G_pruned/timeline_RevEnergia.json 4
output4_G_pruned/timeline_MyGreenSide.json NO DATA FOR MyGreenSide
output4_G_pruned/timeline_NUTROBrand.json NO DATA FOR NUTROBrand
output4_G_pruned/timeline_EcoEcologia.json NO DATA FOR EcoEcologia
output4_G_pruned/timeline_sciencetargets.json 10
output4_G_pruned/timeline_avoicomunicare.json NO DATA FOR avoicomunicare
output4_G_pruned/timeline_EuropeanGasHub.json 4
output4_G_pruned/timeline_EcoSpiragli.json NO DATA FOR EcoSpiragli
output4_G_pruned/timeline_premiomarzotto.json NO DATA FOR premiomarzotto
output4_G_pruned/timeline_bluerating_com.json NO DATA FOR bluerating_com
output4_G_pruned/timeline_euwidenergie.json 5
output4_G_pruned/timeline_GiaSilvestrini.json 35
output4_G_pruned/timeline_IWEA.json 14
output4_G_pruned/timeline_andrew_ward1.json NO DATA FOR andrew_ward1
output4_G_pruned/timeline_CE_NatGas.json 4
output4_G_pruned/timeline_bryworthi

output4_G_pruned/timeline_canaleenergia.json 62
output4_G_pruned/timeline_SmartEnergyExpo.json 0
output4_G_pruned/timeline_fotovoltaicinfo.json NO DATA FOR fotovoltaicinfo
output4_G_pruned/timeline_ValoreSociale.json NO DATA FOR ValoreSociale
output4_G_pruned/timeline_Earth_NewsRT.json NO DATA FOR Earth_NewsRT
output4_G_pruned/timeline_Envirocruit.json NO DATA FOR Envirocruit
output4_G_pruned/timeline_jyn8101.json NO DATA FOR jyn8101
output4_G_pruned/timeline_stefanobesana.json NO DATA FOR stefanobesana
output4_G_pruned/timeline_markettara.json NO DATA FOR markettara
output4_G_pruned/timeline_G20Climate.json 0
output4_G_pruned/timeline_AfricaEUEnergy.json 0
output4_G_pruned/timeline_ap_legambiente.json 46
output4_G_pruned/timeline_dariussnieckus.json 33
output4_G_pruned/timeline_IPIECA.json 0
output4_G_pruned/timeline_windbizfactor.json NO DATA FOR windbizfactor
output4_G_pruned/timeline_Urbact_IT.json NO DATA FOR Urbact_IT
output4_G_pruned/timeline_EarthDayItalia.json NO DATA FOR Eart

output4_G_pruned/timeline_wefenergy.json 3
output4_G_pruned/timeline_amicidellaterra.json NO DATA FOR amicidellaterra
output4_G_pruned/timeline_LegambienteUmbr.json NO DATA FOR LegambienteUmbr
output4_G_pruned/timeline_investtrentino.json NO DATA FOR investtrentino
output4_G_pruned/timeline_greengraffiti.json NO DATA FOR greengraffiti
output4_G_pruned/timeline_Comonext_.json NO DATA FOR Comonext_
output4_G_pruned/timeline__amblav.json NO DATA FOR _amblav
output4_G_pruned/timeline_EnergyViewpoint.json 0
output4_G_pruned/timeline_RANRoberto.json NO DATA FOR RANRoberto
output4_G_pruned/timeline_rvlandberg.json 0
output4_G_pruned/timeline_AigetEnergia.json NO DATA FOR AigetEnergia
output4_G_pruned/timeline_FORESIGHTdk.json 5
output4_G_pruned/timeline_kaitaniemi.json NO DATA FOR kaitaniemi
output4_G_pruned/timeline_aristotele30.json NO DATA FOR aristotele30
output4_G_pruned/timeline_PicloEnergy.json 0
output4_G_pruned/timeline_EnvComm_ACT.json NO DATA FOR EnvComm_ACT
output4_G_pruned/timeli

...

output4_G_pruned/timeline_LeoBrincat.json 27
output4_G_pruned/timeline_klemenzum.json NO DATA FOR klemenzum
output4_G_pruned/timeline_AdinaValean.json 2
output4_G_pruned/timeline_EltisPortal.json 24
output4_G_pruned/timeline_EUKN_EGTC.json NO DATA FOR EUKN_EGTC
output4_G_pruned/timeline_Catalonia_EU.json 1
output4_G_pruned/timeline_euobs.json 159
output4_G_pruned/timeline_Isabruss.json NO DATA FOR Isabruss
output4_G_pruned/timeline_alexmarshall81.json 0
output4_G_pruned/timeline_TweetWatchEU.json NO DATA FOR TweetWatchEU
output4_G_pruned/timeline_EUReuters.json 0
output4_G_pruned/timeline_InterregCE.json 6
output4_G_pruned/timeline_mgracacarvalho.json 15
output4_G_pruned/timeline_collegeofeurope.json 3
output4_G_pruned/timeline_LiquidGasEurope.json 21
output4_G_pruned/timeline_RegionFrankfurt.json 8
output4_G_pruned/timeline_EuroCoopTeam.json 6
output4_G_pruned/timeline_InnovationCircl.json NO DATA FOR InnovationCircl
output4_G_pruned/timeline_cecimo.json 7
output4_G_pruned/timeline_Ze

output4_G_pruned/timeline_EUDefenceAgency.json 4
output4_G_pruned/timeline_solinas_diego.json 26
output4_G_pruned/timeline_Villa_Vigoni.json NO DATA FOR Villa_Vigoni
output4_G_pruned/timeline_MStoppard.json 1
output4_G_pruned/timeline_CopernicusECMWF.json 34
output4_G_pruned/timeline_MalmstromEU.json 5
output4_G_pruned/timeline_paola_tamma.json 0
output4_G_pruned/timeline_CopernicusEU.json 151
output4_G_pruned/timeline_TheNewEuropean.json 0
output4_G_pruned/timeline_ProgressSoc.json NO DATA FOR ProgressSoc
output4_G_pruned/timeline_tilt_green.json 0
output4_G_pruned/timeline_iiea.json 6
output4_G_pruned/timeline_martadassu.json 2
output4_G_pruned/timeline_Aleks_Tomczak.json NO DATA FOR Aleks_Tomczak
output4_G_pruned/timeline_Labour4EU.json NO DATA FOR Labour4EU
output4_G_pruned/timeline_Remain_Labour.json NO DATA FOR Remain_Labour
output4_G_pruned/timeline_Andrew_Adonis.json 11
output4_G_pruned/timeline_NigliaF.json 0
output4_G_pruned/timeline_neilmakaroff.json 1
output4_G_pruned/timel

output4_G_pruned/timeline_EU_IPO.json 10
output4_G_pruned/timeline_EPOorg.json NO DATA FOR EPOorg
output4_G_pruned/timeline_CalaveraRuiz.json 8
output4_G_pruned/timeline_TNavracsicsEU.json 0
output4_G_pruned/timeline_mariannethyssen.json 0
output4_G_pruned/timeline_VisegradInsight.json 12
output4_G_pruned/timeline_SMuresan.json 10
output4_G_pruned/timeline_SwedeninEU.json 0
output4_G_pruned/timeline_MineIntegrimit.json NO DATA FOR MineIntegrimit
output4_G_pruned/timeline_CENAA_.json NO DATA FOR CENAA_
output4_G_pruned/timeline_VivianeRedingEU.json 3
output4_G_pruned/timeline_LSEEuroppblog.json 2
output4_G_pruned/timeline_LSEE_LSE.json NO DATA FOR LSEE_LSE
output4_G_pruned/timeline_Elih_Med.json 0
output4_G_pruned/timeline_EP_Economics.json 0
output4_G_pruned/timeline_TTIPBeware.json NO DATA FOR TTIPBeware
output4_G_pruned/timeline_EUintheUS.json 12
output4_G_pruned/timeline_EUROFER_eu.json 0
output4_G_pruned/timeline_FutureLabEurope.json NO DATA FOR FutureLabEurope
output4_G_pruned/tim

output4_G_pruned/timeline_e_c_c_e.json 21
output4_G_pruned/timeline_CarvalhoLopo.json 0
output4_G_pruned/timeline_EuroCelt.json 8
output4_G_pruned/timeline_LdeRichemont.json 3
output4_G_pruned/timeline_EDIC_uni_lu.json 2
output4_G_pruned/timeline_CarmenGrlica.json 0
output4_G_pruned/timeline_NewTechAqua.json 0
output4_G_pruned/timeline_CORE_Organic.json 0
output4_G_pruned/timeline_EASN_Conference.json 0
output4_G_pruned/timeline_Nathalie_L_EU.json 0
output4_G_pruned/timeline_CEIBois.json 3
output4_G_pruned/timeline_n_panozzo.json 6
output4_G_pruned/timeline_ECA_aisbl.json 0
output4_G_pruned/timeline_SeldiaEU.json 5
output4_G_pruned/timeline_joasialuzak.json 2
output4_G_pruned/timeline_medtecheurope.json 1
output4_G_pruned/timeline_MINDeSEA.json 10
output4_G_pruned/timeline_EHCE_.json 20
output4_G_pruned/timeline_Euro_Sea.json 5
output4_G_pruned/timeline_PolicyCULT.json 2
output4_G_pruned/timeline_charm_eu.json 0
output4_G_pruned/timeline_YERUN_EU.json 6
output4_G_pruned/timeline_EuroSh

output4_G_pruned/timeline_vektom.json 11
output4_G_pruned/timeline_ecrgroup.json 2
output4_G_pruned/timeline_NEMOoffice.json 6
output4_G_pruned/timeline_JCIEurope.json 0
output4_G_pruned/timeline_ERDF_Enhance.json 16
output4_G_pruned/timeline_eyemobilitas.json 1
output4_G_pruned/timeline_AndyZynga.json 7
output4_G_pruned/timeline_yen_lam13.json 3
output4_G_pruned/timeline_EAHLaw.json 6
output4_G_pruned/timeline_emsa_europe.json 0
output4_G_pruned/timeline_CEOM_Brussels.json NO DATA FOR CEOM_Brussels
output4_G_pruned/timeline_Astro_Alex.json 0
output4_G_pruned/timeline_cematce.json 0
output4_G_pruned/timeline_EDCTP.json 7
output4_G_pruned/timeline_marianne_alfsen.json 0
output4_G_pruned/timeline_IoT4Industry18.json 0
output4_G_pruned/timeline_LifeF4FCrete.json 0
output4_G_pruned/timeline_PaolaPepeEU.json 1
output4_G_pruned/timeline_URBACT_IE.json 3
output4_G_pruned/timeline_ElisabethMagnes.json 2
output4_G_pruned/timeline_EvaHernandezWWF.json 0
output4_G_pruned/timeline_ReinekeEG4A.json

In [79]:
for k in tweets_ht.keys():
    print(k,tweets_ht[k])
    break

falckrenewables ['#renewables']


In [80]:
len(tweets_ht.keys())

3112

In [81]:
cnt = 0
for k,v in tweets_ht.items():
    cnt+=len(v)
print(cnt)

15460


In [82]:
tweets_ht_partitioned = {}

for k,v in tweets_ht.items():
    if users_comm[k] in tweets_ht_partitioned.keys():
        tmp = tweets_ht_partitioned[users_comm[k]]
        tmp.append(v)
        tweets_ht_partitioned[users_comm[k]] = tmp
    else:
        tweets_ht_partitioned[users_comm[k]] = [v]

In [83]:
tweets_ht_partitioned.keys()

dict_keys([0, 5, 3, 1, 2, 6, 4, 7])

In [91]:
tweets_ht_partitioned_as_strings = {}

# translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

# for k,v in test_dict.items():
for k,v in tweets_ht_partitioned.items():
    s = ""
    for i in v: # v e' la lista di stringhe di un certo utente
        s += ' '.join(i)
        s += ' '
    s = s.lower()

    tweets_ht_partitioned_as_strings[k] = s
    
# tweets_partitioned_as_strings

In [84]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words=stop_words_it+stop_words_en, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<>:6: DeprecationWarning: invalid escape sequence \-
<ipython-input-84-cde1dcb87f26>:6: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


In [113]:
# TOPICS OF ALL COMMUNITIES

panel_to_save_ht_coll = {}

for k in tweets_ht_partitioned.keys():
    data_vectorized = vectorizer.fit_transform(tweets_ht_partitioned_as_strings[k].split())
    
    NUM_TOPICS = 3

    # Build a Latent Dirichlet Allocation Model
    lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=50, learning_method='online')
    lda_Z = lda_model.fit_transform(data_vectorized)

    # IL NUMERO DI DOCUMENTI QUI INTENDE IL NUMERO DI PAROLE
    print("NumDocuments,NumTopics:",lda_Z.shape)  # (NO_DOCUMENTS, NO_TOPICS) 

    topics = {}
    for idx, topic in enumerate(lda_model.components_):
        topics[idx] = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-NUM_TOPICS - 1:-1]]

    panel_to_save = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

    panel_to_save_ht_coll[k] = panel_to_save

NumDocuments,NumTopics: (1631, 3)
NumDocuments,NumTopics: (2447, 3)
NumDocuments,NumTopics: (1284, 3)
NumDocuments,NumTopics: (1309, 3)
NumDocuments,NumTopics: (2345, 3)
NumDocuments,NumTopics: (2111, 3)
NumDocuments,NumTopics: (3983, 3)
NumDocuments,NumTopics: (350, 3)


In [114]:
panel_to_save_ht_coll[0]

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  41.353220
1      -387.996002  -373.316162       2        1  29.547664
0     -2184.745605   -60.565800       3        1  29.099116, topic_info=   Category        Freq             Term       Total  loglift  logprob
33  Default  110.000000  renewableenergy  110.000000  30.0000  30.0000
37  Default  117.000000            solar  117.000000  29.0000  29.0000
4   Default   68.000000      cleanenergy   68.000000  28.0000  28.0000
34  Default   59.000000       renewables   59.000000  27.0000  27.0000
23  Default   38.000000            irena   38.000000  26.0000  26.0000
..      ...         ...              ...         ...      ...      ...
40   Topic3    0.371360       solarpower   27.447856  -3.0684  -6.4103
34   Topic3    0.371359       renewables   59.112633  -3.8356  -6.4103
37   Topic3    0.371336            solar  117.345797  -4.5213  -6.4103
26   Topic3    0.369678      marsexpress    9.580618  -2.0204  -6.4148
39   Topic3    0.369601     solarorbiter   13.256139  -2.3453  -6.4150

[141 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
0         2  0.978315             africa
1         3  0.957932             amazon
2         3  0.985500          australia
3         2  0.914936                ces
4         2  0.996982        cleanenergy
5         1  0.940859            climate
6         1  0.793921      climateaction
7         3  0.948322      climatechange
8         2  0.856798      climatestrike
9         3  0.955375           colorado
10        2  0.944589        commodities
11        1  0.940589             energy
12        2  0.947996  energyforprogress
13        3  0.930721   energytransition
14        1  0.963053   fridaysforfuture
15        3  0.950793          fukushima
16        2  0.936725                gas
17        1  0.870638        globalgoals
18        3  0.907874         gruppohera
19        2  0.848608          heragroup
20        3  0.928930           hydrogen
21        2  0.962200         hydropower
22        2  0.975049               iran
23        1  0.968947              irena
24        1  0.960676              japan
25        2  0.947996           lifeison
26        2  0.939397        marsexpress
27        3  0.957281     missioncontrol
28        3  0.981253       nationallabs
29        1  0.796800         naturalgas
30        1  0.946746                oil
31        2  0.940799               oott
32        3  0.945659             poland
33        3  0.995004    renewableenergy
34        1  0.981178         renewables
35        1  0.805095       schoolstrike
36        2  0.944115               soae
37        1  0.997053              solar
38        2  0.979066        solarenergy
39        2  0.980678       solarorbiter
40        1  0.983683         solarpower
41        2  0.905248          stopadani
42        2  0.952830     sustainability
43        1  0.882197      todayinenergy
44        3  0.955400           virginia
45        1  0.776650               wind, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [116]:
panel_to_save_ht_coll[1]

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  36.879617
0      -387.996002  -373.316162       2        1  32.753715
1     -2184.745605   -60.565800       3        1  30.366668, topic_info=   Category       Freq           Term      Total  loglift  logprob
7   Default  27.000000  digitalmarket  27.000000  27.0000  27.0000
10  Default  16.000000    eugreendeal  16.000000  26.0000  26.0000
2   Default  17.000000         brexit  17.000000  25.0000  25.0000
9   Default  14.000000         energy  14.000000  24.0000  24.0000
13  Default  10.000000          malta  10.000000  23.0000  23.0000
..      ...        ...            ...        ...      ...      ...
1    Topic3   0.304902      australia   8.309956  -2.1134  -5.4340
4    Topic3   0.304337  climatechange   9.766312  -2.2767  -5.4359
9    Topic3   0.304830         energy  14.166493  -2.6471  -5.4343
2    Topic3   0.304426         brexit  17.532361  -2.8616  -5.4356
7    Topic3   0.304812  digitalmarket  27.456899  -3.3089  -5.4343

[108 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
0         2  0.911583      agriculture
1         1  0.962701        australia
2         2  0.969636           brexit
3         2  0.911543         brussels
4         2  0.921535    climatechange
5         1  0.873104             coal
6         1  0.918478          digital
7         1  0.983359    digitalmarket
8         3  0.869001        emissioni
9         1  0.988247           energy
10        3  0.979421      eugreendeal
11        2  0.936311            euipo
12        3  0.911003          gennaio
13        3  0.909292            malta
14        2  0.960033     mediterraneo
15        1  0.808061      monopattini
16        2  0.937583       publiacqua
17        1  0.787737  regionalaffairs
18        2  0.926403  renewableenergy
19        3  0.944730         research
20        1  0.872392           retail
21        2  0.845560      ruraleurope
22        3  0.912220             smog
23        3  0.971816            trade
24        2  0.846816       utilitalia
25        1  0.815999            water
26        3  0.803248      womeninwind, R=27, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [77]:
# RICORDATI DI CONTROLLARE IL PROBLEMA DEL DISALLINEAMENTO TRA G_pruned_nodes_final.csv E GLI screen_name EFFETTIVAMENTE CARICATI IN QUESTO NOTEBOOK!!!